In [7]:
import random

In [8]:
class EllipticCurve:
   
    
    def __init__(self, a, b, p):
        self.a = a
        self.b = b
        self.p = p
        
    def is_point_on_curve(self, point):
       
        if point is None:  # Point at infinity
            return True
        x, y = point
        left = (y * y) % self.p
        right = (x**3 + self.a * x + self.b) % self.p
        return left == right
    
    def __str__(self):
        return f"y^2 = x^3 + {self.a}x + {self.b} (mod {self.p})"


In [9]:
class Point:
    
    
    def __init__(self, curve, x, y):
        self.curve = curve
        self.x = x % curve.p
        self.y = y % curve.p
        
        # Verify point is on the curve
        if not curve.is_point_on_curve((self.x, self.y)):
            raise ValueError(f"Point ({x}, {y}) is not on the curve {curve}")
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.curve == other.curve
    
    def __neg__(self):
       
        return Point(self.curve, self.x, -self.y % self.curve.p)
    
    def __add__(self, other):
       
        if self.curve != other.curve:
            raise ValueError("Points are on different curves")
        
        # Handle point at infinity
        if self is None:
            return other
        if other is None:
            return self
        
        # Case 1: self and other are negatives of each other
        if self.x == other.x and self.y != other.y:
            return None  # Point at infinity
        
        # Case 2: self == other (point doubling)
        if self == other:
            # Calculate slope for point doubling
            numerator = (3 * self.x * self.x + self.curve.a) % self.curve.p
            denominator = (2 * self.y) % self.curve.p
            # Modular inverse of denominator
            inv_denominator = pow(denominator, -1, self.curve.p)
            m = (numerator * inv_denominator) % self.curve.p
        else:
            # Case 3: self != other (point addition)
            numerator = (other.y - self.y) % self.curve.p
            denominator = (other.x - self.x) % self.curve.p
            # Modular inverse of denominator
            inv_denominator = pow(denominator, -1, self.curve.p)
            m = (numerator * inv_denominator) % self.curve.p
        
        # Calculate new point coordinates
        x3 = (m * m - self.x - other.x) % self.curve.p
        y3 = (m * (self.x - x3) - self.y) % self.curve.p
        
        return Point(self.curve, x3, y3)
    
    def __mul__(self, scalar):
        
        if scalar == 0:
            return None  # Point at infinity
        
        result = None
        addend = self
        
        # Binary expansion method for scalar multiplication
        while scalar:
            if scalar & 1:
                if result is None:
                    result = addend
                else:
                    result = result + addend
            addend = addend + addend
            scalar >>= 1
            
        return result
    
    def __rmul__(self, scalar):
      
        return self * scalar
    
    def __str__(self):
        return f"({self.x}, {self.y})"

In [10]:
class ECCParticipant:
    
    
    def __init__(self, name, curve, base_point):
        self.name = name
        self.curve = curve
        self.base_point = base_point
        self.private_key = None
        self.public_key = None
        self.shared_secret = None
        
    def generate_key_pair(self):
        
        
        self.private_key = random.randint(1, self.curve.p - 1)
        
        
        self.public_key = self.private_key * self.base_point
        
        return self.private_key, self.public_key
    
    def compute_shared_secret(self, other_public_key):
        
        if other_public_key is None:
            raise ValueError("Other participant's public key is None")
        
       
        self.shared_secret = self.private_key * other_public_key
        return self.shared_secret
    
    def __str__(self):
        return f"{self.name}: Private Key = {self.private_key}, Public Key = {self.public_key}"

In [11]:
def main():
   
    a = 2
    b = 3
    p = 97 
    
    curve = EllipticCurve(a, b, p)
    print(f"a) Elliptic curve defined: {curve}")
    
    
    base_x, base_y = 3, 6
    G = Point(curve, base_x, base_y)
    print(f"b) Base point G = {G}")
    print(f"   Verification: {base_y}^2 = {base_y**2}, {base_x}^3 + {a}*{base_x} + {b} = {base_x**3 + a*base_x + b}")
    print()
    
    
    tim = ECCParticipant("Tim", curve, G)
    tim_private, tim_public = tim.generate_key_pair()
    print(f"c) {tim.name}'s Key Pair:")
    print(f"   Private Key: {tim_private}")
    print(f"   Public Key: {tim_public}")
    print()
    
    # d) Generate Stephen's private key and compute his public key
    stephen = ECCParticipant("Stephen", curve, G)
    stephen_private, stephen_public = stephen.generate_key_pair()
    print(f"d) {stephen.name}'s Key Pair:")
    print(f"   Private Key: {stephen_private}")
    print(f"   Public Key: {stephen_public}")
    print()
    
    
    print("e) Computing shared secrets:")
    
    
    tim_shared = tim.compute_shared_secret(stephen_public)
    print(f"   Tim's shared secret: {tim_shared}")
    
   
    stephen_shared = stephen.compute_shared_secret(tim_public)
    print(f"   Stephen's shared secret: {stephen_shared}")
    print()
    
    # f) Display all keys and verify they match
    print("f) Verification:")
    print(f"   Tim's private key: {tim_private}")
    print(f"   Tim's public key: {tim_public}")
    print(f"   Stephen's private key: {stephen_private}")
    print(f"   Stephen's public key: {stephen_public}")
    print(f"   Tim's shared secret: {tim_shared}")
    print(f"   Stephen's shared secret: {stephen_shared}")
    
    
    if tim_shared == stephen_shared:
        print(f"\n  SUCCESS: Both participants have the same shared secret!")
        print(f"   Shared secret point: {tim_shared}")
    else:
        print(f"\n  ERROR: Shared secrets do not match!")




In [12]:
if __name__ == "__main__":
    main()

a) Elliptic curve defined: y^2 = x^3 + 2x + 3 (mod 97)
b) Base point G = (3, 6)
   Verification: 6^2 = 36, 3^3 + 2*3 + 3 = 36

c) Tim's Key Pair:
   Private Key: 79
   Public Key: (3, 91)

d) Stephen's Key Pair:
   Private Key: 56
   Public Key: (3, 6)

e) Computing shared secrets:
   Tim's shared secret: (3, 91)
   Stephen's shared secret: (3, 91)

f) Verification:
   Tim's private key: 79
   Tim's public key: (3, 91)
   Stephen's private key: 56
   Stephen's public key: (3, 6)
   Tim's shared secret: (3, 91)
   Stephen's shared secret: (3, 91)

  SUCCESS: Both participants have the same shared secret!
   Shared secret point: (3, 91)
